In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer

%matplotlib inline
%reload_ext autoreload
%autoreload 2

from src import *

In [2]:
# Load resources from HF
hf_model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained(
    "openai-community/gpt2", clean_up_tokenization_spaces=False
)

In [3]:
# Load weights in model
num_heads = 12
embed_dim = 768
context_len = 1024

model = modules.GPT2(tokenizer.vocab_size, embed_dim, context_len, num_heads)
model.load_state_dict(hf_model.state_dict())

<All keys matched successfully>

In [4]:
prompt = "Emergency Broadcast! At 1200, the Earth"

g = torch.Generator().manual_seed(42)
hf_completions = pipeline.generate_completion(
    prompt, tokenizer, hf_model, generator=g, loading_bar_prefix="HF Completions"
)
g = torch.Generator().manual_seed(42)
completions = pipeline.generate_completion(
    prompt, tokenizer, model, generator=g, loading_bar_prefix="Our Completions"
)

completion_padding = " " * 11
for i, (hf_completion, completion) in enumerate(zip(hf_completions, completions)):
    print(f"\nCompletion {i}:\n============")
    print(f"Got      : {completion.replace('\n\n', '\n' + completion_padding)}")
    print(completion_padding + "---------------------------------------------")
    print(f"Expected : {hf_completion.replace('\n\n', '\n' + completion_padding)}")

Our Completions: 100%|██████████| 40/40 [00:04<00:00,  8.43it/s]


Completion 0:
Got      : Emergency Broadcast! At 1200, the Earth's 912 hour-2900/2000 It's Salisbury! A car his shaken hands. No clue who or what he was, but this Cab mess! * LANGUAGE: Wis
           ---------------------------------------------
Expected : Emergency Broadcast! At 1200, the Earth's 912 hour-2900/2000 It's Salisbury! A car his shaken hands. No clue who or what he was, but this Cab mess! * LANGUAGE: Wis

Completion 1:
Got      : Emergency Broadcast! At 1200, the Earth's furnace has failed! In my Yamato Colonies and several other states of smelting were toppled under the swayy clouds. But I was saved by the best minds within Seventh Saturdays at 1000.
           ---------------------------------------------
Expected : Emergency Broadcast! At 1200, the Earth's furnace has failed! In my Yamato Colonies and several other states of smelting were toppled under the swayy clouds. But I was saved by the best minds within Seventh Saturdays at 1000.

Completion 2:
Got      : Emerge